# Lab 19a: Cloud Security Fundamentals

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab19a_cloud_security_fundamentals.ipynb)

Understanding cloud infrastructure and security before diving into AI-powered cloud detection.

## Learning Objectives

1. Understand core cloud concepts (IaaS, PaaS, SaaS)
2. Navigate AWS, Azure, and GCP security logging
3. Understand IAM (Identity and Access Management) across clouds
4. Know key security services and their purposes

## Part 1: Cloud Service Models

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                         CLOUD SERVICE MODELS                                 │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│   On-Premises        IaaS              PaaS              SaaS               │
│   ───────────        ────              ────              ────               │
│                                                                             │
│   Applications      Applications      Applications      ████████████       │
│   Data              Data              Data              ████████████       │
│   Runtime           Runtime           ████████████      ████████████       │
│   Middleware        Middleware        ████████████      ████████████       │
│   OS                OS                ████████████      ████████████       │
│   Virtualization    ████████████      ████████████      ████████████       │
│   Servers           ████████████      ████████████      ████████████       │
│   Storage           ████████████      ████████████      ████████████       │
│   Networking        ████████████      ████████████      ████████████       │
│                                                                             │
│   You Manage        ████████████ = Provider Manages                         │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

| Model | Examples | You're Responsible For |
|-------|----------|------------------------|
| **IaaS** | EC2, Azure VM, GCE | OS patches, app security, data encryption |
| **PaaS** | Lambda, App Engine | Application code, data, IAM |
| **SaaS** | Microsoft 365, Salesforce | User access, data classification |

## Part 2: The Big Three - Service Mapping

| Category | AWS | Azure | GCP |
|----------|-----|-------|-----|
| **Compute** | EC2 | Virtual Machines | Compute Engine |
| **Serverless** | Lambda | Functions | Cloud Functions |
| **Containers** | EKS, ECS | AKS | GKE |
| **Storage** | S3 | Blob Storage | Cloud Storage |
| **IAM** | IAM | Entra ID (Azure AD) | Cloud IAM |
| **Logging** | CloudTrail | Activity Log | Cloud Audit Logs |
| **SIEM** | Security Lake | Sentinel | Chronicle |

## Part 3: Cloud Logging for Security

### AWS CloudTrail Event Structure

```json
{
    "eventVersion": "1.08",
    "userIdentity": {
        "type": "IAMUser",
        "principalId": "AIDAEXAMPLE",
        "arn": "arn:aws:iam::123456789012:user/alice",
        "accountId": "123456789012",
        "userName": "alice"
    },
    "eventTime": "2024-01-15T10:30:00Z",
    "eventSource": "s3.amazonaws.com",
    "eventName": "GetObject",
    "sourceIPAddress": "203.0.113.50"
}
```

In [ ]:
import json
from dataclasses import dataclass
from typing import List, Dict, Optional
from datetime import datetime

@dataclass
class CloudTrailEvent:
    """Parsed CloudTrail event."""
    event_time: str
    event_source: str
    event_name: str
    user_identity: Dict
    source_ip: str
    error_code: Optional[str] = None


class CloudTrailParser:
    """Parse and analyze CloudTrail events."""
    
    # High-risk API calls to monitor
    HIGH_RISK_EVENTS = [
        "CreateUser",
        "CreateAccessKey",
        "AttachUserPolicy",
        "PutBucketPolicy",
        "StopLogging",
        "DeleteTrail",
        "AuthorizeSecurityGroupIngress",
    ]
    
    def parse_event(self, raw_event: dict) -> CloudTrailEvent:
        """Parse a raw CloudTrail event."""
        return CloudTrailEvent(
            event_time=raw_event.get("eventTime", ""),
            event_source=raw_event.get("eventSource", ""),
            event_name=raw_event.get("eventName", ""),
            user_identity=raw_event.get("userIdentity", {}),
            source_ip=raw_event.get("sourceIPAddress", ""),
            error_code=raw_event.get("errorCode"),
        )
    
    def is_high_risk(self, event: CloudTrailEvent) -> bool:
        """Check if event is high-risk."""
        return event.event_name in self.HIGH_RISK_EVENTS
    
    def get_principal(self, event: CloudTrailEvent) -> str:
        """Extract the principal (who did this)."""
        identity = event.user_identity
        if identity.get("type") == "IAMUser":
            return identity.get("userName", "Unknown")
        elif identity.get("type") == "AssumedRole":
            return identity.get("arn", "Unknown")
        elif identity.get("type") == "Root":
            return "ROOT ACCOUNT"
        return identity.get("type", "Unknown")

# Example usage
parser = CloudTrailParser()

sample_event = {
    "eventTime": "2024-01-15T10:30:00Z",
    "eventSource": "iam.amazonaws.com",
    "eventName": "CreateAccessKey",
    "userIdentity": {"type": "IAMUser", "userName": "alice"},
    "sourceIPAddress": "203.0.113.50"
}

event = parser.parse_event(sample_event)
print(f"Event: {event.event_name}")
print(f"Principal: {parser.get_principal(event)}")
print(f"High Risk: {parser.is_high_risk(event)}")

## Part 4: IAM Fundamentals

### The IAM Mental Model

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                         IAM FUNDAMENTALS                                     │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│   PRINCIPAL          ACTION              RESOURCE          CONDITION        │
│   ─────────          ──────              ────────          ─────────        │
│   WHO is making      WHAT are they       ON WHAT           UNDER WHAT       │
│   the request?       trying to do?       resource?         conditions?      │
│                                                                             │
│   • User             • s3:GetObject      • arn:aws:s3:::   • IP range       │
│   • Role             • ec2:RunInstances    my-bucket/*     • Time of day    │
│   • Service          • iam:CreateUser    • arn:aws:ec2:    • MFA required   │
│                                            ...                               │
│                                                                             │
│   EVALUATION: Allow if ANY policy grants access AND no policy denies        │
│               Explicit Deny > Explicit Allow > Implicit Deny                │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
@dataclass
class IAMFinding:
    """Security finding related to IAM."""
    severity: str
    title: str
    description: str
    resource: str
    recommendation: str


class IAMAnalyzer:
    """Analyze IAM configurations for security issues."""
    
    def check_overly_permissive(self, policy: dict) -> List[IAMFinding]:
        """Check for overly permissive policies."""
        findings = []
        
        for statement in policy.get("Statement", []):
            action = statement.get("Action", [])
            resource = statement.get("Resource", [])
            
            if isinstance(action, str):
                action = [action]
            if isinstance(resource, str):
                resource = [resource]
            
            # Check for admin access
            if "*" in action and "*" in resource:
                findings.append(IAMFinding(
                    severity="CRITICAL",
                    title="Full Admin Access",
                    description="Policy grants * actions on * resources",
                    resource=str(resource),
                    recommendation="Apply least privilege - limit to specific actions and resources"
                ))
            
            # Check for wildcard actions
            if any(":*" in a for a in action):
                findings.append(IAMFinding(
                    severity="HIGH",
                    title="Wildcard Actions",
                    description=f"Policy grants wildcard actions: {action}",
                    resource=str(resource),
                    recommendation="Specify exact actions needed"
                ))
        
        return findings

# Example
analyzer = IAMAnalyzer()

risky_policy = {
    "Statement": [{
        "Effect": "Allow",
        "Action": "*",
        "Resource": "*"
    }]
}

findings = analyzer.check_overly_permissive(risky_policy)
for finding in findings:
    print(f"[{finding.severity}] {finding.title}")
    print(f"  Description: {finding.description}")
    print(f"  Recommendation: {finding.recommendation}")

## Part 5: Common Cloud Attack Patterns

| Attack Pattern | Indicators | Detection |
|----------------|------------|------------|
| **Credential Theft** | New access keys, unusual IPs | Monitor CreateAccessKey, track IP anomalies |
| **Privilege Escalation** | Policy changes, role assumptions | Monitor IAM changes, track AssumeRole |
| **Data Exfiltration** | Large S3 downloads, unusual egress | Monitor GetObject, network flow logs |
| **Resource Hijacking** | New instances, crypto mining | Monitor RunInstances, CPU anomalies |
| **Persistence** | New users, modified policies | Monitor CreateUser, PutUserPolicy |

In [ ]:
class CloudThreatDetector:
    """Detect common cloud attack patterns."""
    
    def __init__(self):
        self.persistence_events = [
            "CreateUser", "CreateRole", "CreateAccessKey",
            "AttachUserPolicy", "AttachRolePolicy"
        ]
        self.defense_evasion_events = [
            "StopLogging", "DeleteTrail", "UpdateTrail",
            "DeleteFlowLogs", "DisableAlarmActions"
        ]
        self.exfiltration_events = [
            "GetObject", "CopyObject", "CreateSnapshot"
        ]
    
    def classify_event(self, event_name: str) -> str:
        """Classify event by MITRE ATT&CK tactic."""
        if event_name in self.persistence_events:
            return "PERSISTENCE"
        elif event_name in self.defense_evasion_events:
            return "DEFENSE_EVASION"
        elif event_name in self.exfiltration_events:
            return "EXFILTRATION"
        return "UNKNOWN"
    
    def get_severity(self, event_name: str) -> str:
        """Determine event severity."""
        if event_name in self.defense_evasion_events:
            return "CRITICAL"
        elif event_name in self.persistence_events:
            return "HIGH"
        return "MEDIUM"

# Example
detector = CloudThreatDetector()
test_events = ["CreateAccessKey", "StopLogging", "GetObject"]

for event in test_events:
    tactic = detector.classify_event(event)
    severity = detector.get_severity(event)
    print(f"{event}: {tactic} ({severity})")

## Key Takeaways

1. **Shared Responsibility** - Know what you're responsible for vs. the provider
2. **Logging is Critical** - CloudTrail/Activity Log/Audit Logs are your eyes
3. **IAM is Complex** - Least privilege is hard but essential
4. **Multi-Cloud is Reality** - Learn to map concepts across providers

## Next Steps

- **Lab 19**: Cloud Security AI - Build AI-powered cloud detection
- **Lab 09**: Detection Pipeline - Integrate cloud logs into detection